In [ ]:
import sympy as s
from IPython.display import display
from pprint import pprint

In [ ]:
i1,i2,i3,i4,i5,i6 = s.symbols(" ".join([f"x_{{i{i}}}" for i in range(6)]))
j1,j2,j3,j4,j5,j6 = s.symbols(" ".join([f"x_{{j{i}}}" for i in range(6)]))
z1,z2,z3,z4,z5,z6,z7,z8,z9,z10,z11,z12 = s.symbols(" ".join([f"z_{{{i}}}" for i in range(12)]))
theta_i, theta_j, theta_z = s.symbols(r"\theta_i \theta_j \theta_z")
Zij = s.Matrix([
    [z1,z2,z3,z10],
    [z4,z5,z6,z11],
    [z7,z8,z9,z12],
    [0,0,0,1]
])

theta_i

In [ ]:
def cross(a,b,c):
    return s.Matrix([
        [0,-c,b],
        [c,0,-a],
        [-b,a,0]
    ])

def exp(x1,x2,x3,x4,x5,x6):
    theta = s.sqrt(x4*x4 + x5*x5 + x6*x6)
    a = s.sin(theta)/theta
    b = (1 - s.cos(theta))/(theta*theta)
    c = (theta - s.sin(theta))/(theta**3)
    w = cross(x4,x5,x6)
    R = s.eye(3) + a*w + b*w@w

    t = (s.eye(3) + b*w + c*w@w)@s.Matrix([[x1], [x2], [x3]])
    
    mat = s.eye(4)
    mat[:3, :3] = R
    mat[:3, 3] = t

    return mat

def log(T):
    theta = s.acos((T[0, 0] + T[1, 1] + T[2, 2] - 1) / 2)

    if theta == 0:
        return T[0, 3], T[1, 3], T[2, 3], 0, 0, 0

    a = s.sin(theta)/theta
    b = (1 - s.cos(theta))/(theta*theta)

    x4 = 1/(2*a)*(T[2, 1] - T[1, 2])
    x5 = 1/(2*a)*(T[0, 2] - T[2, 0])
    x6 = 1/(2*a)*(T[1, 0] - T[0, 1])
    
    w = cross(x4,x5,x6)
    u = (s.eye(3) - 1/2*w + 1/(theta**2)*(1 - a/(2*b))*w@w)@T[:3, 3]

    return u[0], u[1], u[2], x4, x5, x6

log(exp(0.1,0.2,0.3,0.4,0.5,0.6))

In [ ]:
def invert(T):
    R = T[:3, :3]
    t = T[:3, 3]
    
    mat = s.eye(4)
    mat[:3, :3] = R.T
    mat[:3, 3] = -R.T@t

    return mat

In [ ]:
Xi = exp(i1,i2,i3,i4,i5,i6)
Xj = exp(j1,j2,j3,j4,j5,j6)
Zij = exp(z1,z2,z3,z4,z5,z6)

Xi = Xi.subs(s.sqrt(i4*i4 + i5*i5 + i6*i6), theta_i)
Xj = Xj.subs(s.sqrt(j4*j4 + j5*j5 + j6*j6), theta_j)
Zij = Zij.subs(s.sqrt(z4*z4 + z5*z5 + z6*z6), theta_z)

Xi

In [ ]:
e = log(invert(Zij)@invert(Xi)@Xj)
e[4]

# Check simplification

In [ ]:
Xi = exp(i1,i2,i3,i4,i5,i6)
Xj = exp(j1,j2,j3,j4,j5,j6)
Zij = exp(z1,z2,z3,z4,z5,z6)

In [ ]:
RXi = Xi[:3,:3]
RXj = Xj[:3,:3]
RZij = Zij[:3,:3]
tXi = Xi[:3, 3]
tXj = Xj[:3, 3]
tZij = Zij[:3, 3]

In [ ]:
res1 = invert(Zij)@invert(Xi)@Xj

In [ ]:
res2 = s.eye(4)
res2[:3,:3] = RZij.T@RXi.T@RXj
res2[:3, 3] = RZij.T@(RXi.T@(tXj - tXi) - tZij)

In [ ]:
A,B,C = s.MatrixSymbol("A", 4, 4),s.MatrixSymbol("B", 4, 4),s.MatrixSymbol("C", 4, 4)
A.inv()@B@C.inv()@B.inv()@A

Pretty sure that does give me the correct result

$$Z_{ij}^{-1}X_i^{-1}X_j \equiv [R_z^TR_i^TR_j | R_z^T(R_i^T(t_j - t_i) - t_z)]$$

However how I take the derivative of that with the log function I'm not entirely sure. I guess I can pass it through the log function and then take the elementwise derivative but that sounds rather messy to say the least. Honestly this whole thing is getting rather messy. It would be really nice if there was a way to get take the derivative of the log function and just chain rule it.

[This answer](https://math.stackexchange.com/questions/723262/explicit-proof-of-the-derivative-of-a-matrix-logarithm) suggests that the derivative of the matrix log function is:
$$X'(x)X^{-1}$$

Which I'm going to try use to solve this silly little jacobian. I'm not entirely convinced that its the right way to do it but I'm going to give it a shot anyway. Actually wait - if we look at the dimensions of it - I'm going to end up with a $4\times4$ matrix which definitely isnt the $6\times1$ vector that I would want from this. Damn. Thats not going to work at all. Back to the drawing board I guess. 

$\ominus$

In [ ]:
import numpy as np

def npcross(x):
    return np.array([
        [0,-x[2],x[1]],
        [x[2],0,-x[0]],
        [-x[1],x[0],0]
    ])

def npexp(x):
    theta = np.linalg.norm(x[3:])
    if theta == 0:
        R = np.eye(3)
        t = x[:3]
    else:
        a = np.sin(theta)/theta
        b = (1 - np.cos(theta))/(theta*theta)
        c = (theta - np.sin(theta))/(theta**3)
        w = npcross(x[3:])
        R = np.eye(3) + a*w + b*w@w

        t = (np.eye(3) + b*w + c*w@w)@x[:3]
    
    mat = np.eye(4)
    mat[:3, :3] = R
    mat[:3, 3] = t

    return mat

def nplog(T):
    theta = np.arccos((np.trace(T) - 2) / 2)
    # theta = np.arccos((T[0, 0] + T[1, 1] + T[2, 2] - 1) / 2)

    if theta == 0:
        return T[0, 3], T[1, 3], T[2, 3], 0, 0, 0

    a = np.sin(theta)/theta
    b = (1 - np.cos(theta))/(theta*theta)

    x4 = 1/(2*a)*(T[2, 1] - T[1, 2])
    x5 = 1/(2*a)*(T[0, 2] - T[2, 0])
    x6 = 1/(2*a)*(T[1, 0] - T[0, 1])
    
    w = npcross(np.array([x4,x5,x6]))
    u = (np.eye(3) - 1/2*w + 1/(theta**2)*(1 - a/(2*b))*w@w)@T[:3, 3]

    return u[0], u[1], u[2], x4, x5, x6

np.isclose(nplog(npexp(np.array([0.1,0.2,0.3,0.4,0.5,0.6]))), np.array([0.1,0.2,0.3,0.4,0.5,0.6]))

In [ ]:
def np_adjoint(X):
    """X is a 4x4 matrix not a 6 Vector"""
    mat = np.zeros((6,6))
    mat[:3,:3] = X[:3,:3]
    mat[3:,3:] = X[:3,:3]
    mat[:3,3:] = npcross(X[:3,3])@X[:3,:3]

    return mat.T


In [ ]:
def np_error(Z, Xi, Xj):
    return nplog(np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi))@npexp(Xj))

def np_exp_error(Z, Xi, Xj):
    return np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi))@npexp(Xj)


In [ ]:
def Je_numeric_Xi(Z, Xi, Xj, eps=1E-4, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    gen_error = [np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi)@npexp(g_i*eps))@npexp(Xj) for g_i in generators]

    cols = (1/eps)*np.array([nplog(error_inv@gen_error_i) for gen_error_i in gen_error])

    return cols

a = np.array([0.1,0.2,0.3,0.4,0.5,0.6])
with np.printoptions(precision=4, suppress=True):
    print(Je_numeric_Xi(a,a,a))

In [ ]:
def Je_analytic_Xi(Z, Xi, Xj):
    eXi, eXj = npexp(Xi), npexp(Xj)
    A = np.zeros((6,6))
    A[:3,:3] = (eXj[:3,:3]).T
    A[3:,3:] = (eXj[:3,:3]).T
    A[:3,3:] = -(eXj[:3,:3]).T@npcross(eXj[:3,3])
    # print("A: ")
    # print(A)

    B = np.zeros_like(A)
    B[:3, :3] = eXi[:3,:3]
    B[3:,3:] = eXi[:3,:3]
    B[:3,3:] = npcross(eXi[:3, 3])@eXi[:3,:3]
    # print("\n\nB: ")
    # print(B)

    return -(A@B).T


with np.printoptions(precision=4, suppress=True):
    vals = (np.random.random(6),np.random.random(6),np.random.random(6))
    res_analytic = Je_analytic_Xi(*vals[:])
    res_numeric = Je_numeric_Xi(*vals[:], eps=1E-7)
    print("\n\nResult analytic: ")
    print(res_analytic)
    print("\n\nResult numeric: ")
    print(res_numeric)
    print(np.all(np.isclose(res_analytic, res_numeric)))

In [ ]:
def Je_numeric_Xj(Z, Xi, Xj, eps=1E-4, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    gen_error = [np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi))@(npexp(Xj)@npexp(g_i*eps)) for g_i in generators]

    cols = (1/eps)*np.array([nplog(error_inv@gen_error_i) for gen_error_i in gen_error])

    return cols

a = np.array([0.1,0.2,0.3,0.4,0.5,0.6])
with np.printoptions(precision=4, suppress=True):
    vals = (np.random.random(6),np.random.random(6),np.random.random(6))
    print(Je_numeric_Xj(*vals))

## In vector space
The above, I think, gives the results of the jacobian in the se(3) space rather than the vector space of the exponential coordinates. So what I think I need to do is pre-multiply by the adjoint of the whole error function but I'm not entirely convinced thats the case.

In [ ]:
def Je_numeric_Xi(Z, Xi, Xj, eps=1E-8, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    gen_error = [np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi)@npexp(g_i*eps))@npexp(Xj) for g_i in generators]
    # gen_error = [np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi + g_i*eps))@npexp(Xj) for g_i in generators]


    cols = (1/eps)*np.array([nplog(error_inv@gen_error_i) for gen_error_i in gen_error]).T

    return cols

def Je_analytic_Xi(Z, Xi, Xj):
    eXi, eXj = npexp(Xi), npexp(Xj)
    A = np.zeros((6,6))
    A[:3,:3] = (eXj[:3,:3]).T
    A[3:,3:] = (eXj[:3,:3]).T
    A[:3,3:] = -(eXj[:3,:3]).T@npcross(eXj[:3,3])
    # print("A: ")
    # print(A)

    B = np.zeros_like(A)
    B[:3, :3] = eXi[:3,:3]
    B[3:,3:] = eXi[:3,:3]
    B[:3,3:] = npcross(eXi[:3, 3])@eXi[:3,:3]
    # print("\n\nB: ")
    # print(B)

    adj = np_adjoint(np.linalg.inv(npexp(Z))@np.linalg.inv(npexp(Xi))@npexp(Xj))

    return -A@B


with np.printoptions(precision=4, suppress=True):
    vals = (np.random.random(6),np.random.random(6),np.random.random(6))
    res_analytic = Je_analytic_Xi(*vals[:])
    res_numeric = Je_numeric_Xi(*vals[:], eps=1E-7)
    print("\n\nResult analytic: ")
    print(res_analytic)
    print("\n\nResult numeric: ")
    print(res_numeric)
    # print("\n\nDifference: ")
    # print(res_numeric@np.linalg.inv(res_analytic))
    # print("\n\nAdjoint: ")
    # print(np_adjoint(np.linalg.inv(npexp(vals[0]))@np.linalg.inv(npexp(vals[1]))@npexp(vals[2])))
    # print(np_adjoint(np.linalg.inv(npexp(vals[1]))))

    print(np.isclose(res_analytic, res_numeric))

In [ ]:
def calc_coeffs(theta):
    return (
        np.sin(theta)/theta,
        (1-np.cos(theta))/(theta**2),
        (theta - np.sin(theta))/(theta**3),
        (theta*np.cos(theta) - np.sin(theta))/theta**3,
        (2*np.cos(theta) - 2 + theta*np.sin(theta))/theta**4,
        (3*np.sin(theta) - 2*theta - theta*np.cos(theta))/theta**5
)


def calc_generators(omega):
    w = npcross(omega)
    w2 = w@w
    w_dash = [
        np.array([
            [0,0,0],
            [0,0,-1],
            [0,1,0]
        ]), 
        np.array([
            [0,0,1],
            [0,0,0],
            [-1,0,0]
        ]),
        np.array([
            [0,-1,0],
            [1,0,0],
            [0,0,0]
        ])
    ]
    w2_dash = [x@w + w@x for x in w_dash]

    return w, w2, w_dash, w2_dash

In [ ]:
def calc_jacobian(x):
    theta = np.linalg.norm(x[3:])
    rho = x[:3].reshape((3,1))
    omega = x[3:]
    print(rho.shape)

    coeffs = calc_coeffs(theta)
    generators = calc_generators(x[3:])

    dr = [coeffs[3]*var*generators[0] + coeffs[0]*generators[2] + coeffs[4]*var*generators[1] + coeffs[1]*generators[3] for var in omega]
    dt = [(coeffs[4]*var*generators[0] + coeffs[1]*generators[2] + coeffs[5]*var*generators[1] + coeffs[2]*generators[3])*rho for var in omega]

# calc_jacobian(a)

## Changing Parameterisation
So it seems that exponential coordinates, albeit having the nice lie algebra property, have a big fat singularity at the identity - which is basically where everything starts off at. Obviously that isn't ideal. So I'm going to try using a different parameterisation - a concatenation of a translation vector and the vector part of a quaternion. Roughly:

$$e_{ij}(\bold{x}) = (z_{ij}^{-1} \oplus (x_i^{-1} \oplus x_j))$$

$$x_i \oplus x_j = \begin{pmatrix} q_i(t_j) \\ q_i \dot q_j \end{pmatrix} $$

In [ ]:
from scipy.spatial.transform import Rotation
import quaternion


def make_quat():
    angle = np.random.random()*2*np.pi
    axis = np.random.random(3)-0.5
    axis = axis / np.linalg.norm(axis)

    w = np.cos(angle/2)
    vec = np.sin(angle/2)*axis

    return w, vec

def rotation_from_np_quat(quat):
    return Rotation.from_quat([quat.x, quat.y, quat.z, quat.w])

p_real, p_imag = make_quat()
q_real, q_imag = make_quat()

p_t = np.random.random(3)*2 - 1
q_t = np.random.random(3)*2 - 1

a = Rotation.from_quat(np.array([*p_imag, p_real]))
b = Rotation.from_quat(np.array([*q_imag, q_real]))

p_mat = np.eye(4)
p_mat[:3,:3] = a.as_matrix()
p_mat[:3,3] = p_t

q_mat = np.eye(4)
q_mat[:3,:3] = b.as_matrix()
q_mat[:3,3] = q_t

mat_res = p_mat@q_mat



p, q = np.quaternion(p_real, *p_imag), np.quaternion(q_real, *q_imag)

new_quat = p*q
new_t = (p*np.quaternion(0, *q_t)*p.conjugate()).imag + p_t

new_rot = rotation_from_np_quat(p*q)
quat_res = np.eye(4)
quat_res[:3,:3] = new_rot.as_matrix()
quat_res[:3,3] = new_t

np.isclose(mat_res, quat_res)

In [ ]:
np.quaternion(1,2,3,4).w

## SymPy Quaternion

In [ ]:
def make_s_pose(symbol):
    w,i,j,k,x,y,z = s.symbols(f"{symbol}_w {symbol}_i {symbol}_j {symbol}_k {symbol}_x {symbol}_y {symbol}_z")

    return s.Quaternion(w,i,j,k, True), s.Quaternion(0,x,y,z)

q_z, t_z = make_s_pose("Z")
q_xi, t_xi = make_s_pose("X")
q_yj, t_yj = make_s_pose("Y")

q_xi, t_xi

In [ ]:
def pqp_star_pure_q(p, q):
    
    pw = p.a
    pv = s.Matrix([p.b,p.c,p.d])
    qv = s.Matrix([q.b,q.c,q.d])

    vector = pv.dot(qv)*pv + (pw**2)*qv + 2*pw*pv.cross(qv) - pv.cross(qv).cross(pv)

    return s.Quaternion(0, *vector)

w,a,b,c,x,y,z = s.symbols("p_w p_i p_j p_k q_i q_j q_k")

p = s.Quaternion(w,a,b,c)
q = s.Quaternion(0,x,y,z)
s.simplify(pqp_star_pure_q(p.conjugate(), q) - p.conjugate()*q*p)

In [ ]:
def make_symbols(symbol:str):
    if "{" in symbol:
        w,i,j,k,x,y,z = s.symbols(f"{symbol}w}} {symbol}i}} {symbol}j}} {symbol}k}} {symbol}x}} {symbol}y}} {symbol}z}}")
    else:
        w,i,j,k,x,y,z = s.symbols(f"{symbol}_w {symbol}_i {symbol}_j {symbol}_k {symbol}_x {symbol}_y {symbol}_z")

    q = s.Quaternion(w,i,j,k)
    t = s.Quaternion(0,x,y,z)

    return q, t

def error(zq, zt, xiq, xit, xjq, xjt):
    et = pqp_star_pure_q(zq.conjugate(), pqp_star_pure_q(xiq.conjugate(), xjt)) - pqp_star_pure_q(zq.conjugate(), pqp_star_pure_q(xiq.conjugate(), xit)) - pqp_star_pure_q(zq.conjugate(), zt)
    eq = zq.conjugate()*xiq.conjugate()*xjq

    return s.Matrix([et.b, et.c, et.d, eq.b, eq.c, eq.d])

zq, zt = make_symbols("Z_{ij")
xiq, xit = make_symbols("X_{i")
xjq, xjt = make_symbols("X_{j")

e = error(zq,zt,xiq,xit,xjq,xjt)
et2 = zq.conjugate()*xiq.conjugate()*(xjt - xit)*xiq*zq - zq.conjugate()*zt*zq
eq2 = zq.conjugate()*xiq.conjugate()*xjq
e2 = s.Matrix([et2.b, et2.c, et2.d, eq2.b, eq2.c, eq2.d])
s.simplify(e - e2)

## Results for PGO jacobian on new Parameterisation
1. Analytic jacobian of Xi
2. Analytic jacobian of Xj
3. Analytic jacobian of the projection into the manifold


In [ ]:
def Je_analytic_Xi(Zq, Zt, Xiq, Xit, Xjq, Xjt):
    # et = Zq.conjugate()*(Xiq.conjugate()*(Xjt - Xit)*Xiq - Zt)*Zq
    # eq = Zq.conjugate()*Xiq.conjugate()*Xjt 
    Jt = translation_error_derivative(Zq.conjugate()*Xiq.conjugate(), Xit)

    xjt_part = translation_rotation_error_derivative(Xiq.conjugate(), Zq.conjugate(), Xjt)
    xit_part = translation_rotation_error_derivative(Xiq.conjugate(), Zq.conjugate(), Xit)

    JetXq = xjt_part - xit_part

    JeqXq = s.Matrix([[(Zq.conjugate()*dpv*Xjq).b,(Zq.conjugate()*dpv*Xjq).c,(Zq.conjugate()*dpv*Xjq).d] for dpv in [s.Quaternion(x[0], -x[1], -x[2], -x[3]) for x in np.eye(4)]])

    res = s.zeros(6,7)

    print(Jt.shape)

    res[:3,:3] = -Jt.T
    res[:3, 3:] = JetXq.T
    res[3:, 3:] = JeqXq.T

    return res

def translation_error_derivative(q, xt):
    pw = q.a
    pv = s.Matrix([q.b, q.c, q.d]) # q
    qv = s.Matrix([xt.b, xt.c, xt.d]) # xt

    Jt = [pv.dot(dqv)*pv + pw**2*dqv + 2*pw*pv.cross(dqv) - pv.cross(dqv).cross(pv) for dqv in [s.Matrix([x[0], x[1], x[2]]) for x in np.eye(3)]]
    
    return s.Matrix([[x[0,0], x[1,0], x[2,0]] for x in Jt])

def translation_rotation_error_derivative(xiq, zq, q):
    pw = xiq.a
    pv = s.Matrix([xiq.b, xiq.c, xiq.d]) # -xiq
    qv = s.Matrix([q.b, q.c, q.d]) # q
    
    dqvs = [2*pw*qv + 2*pv.cross(qv)]

    for dpv in [s.Matrix([x[0], x[1], x[2]]) for x in -np.eye(3)]:
        dqvs.append(dpv.dot(qv)*pv + pv.dot(qv)*dpv + 2*pw*dpv.cross(qv) - dpv.cross(qv).cross(pv) - pv.cross(qv).cross(dpv))

    pw = zq.a
    pv = s.Matrix([zq.b, zq.c, zq.d]) # -zq
    q = xiq.conjugate()*q*xiq
    qv = s.Matrix([q.b, q.c, q.d]) # q

    res_vec = []
    for dqv in dqvs:
        res_vec.append(pv.dot(dqv)*pv + pw**2*dqv + 2*pw*pv.cross(dqv) - pv.cross(dqv).cross(pv))

    return s.Matrix([x.values() for x in res_vec])

je = Je_analytic_Xi(zq,zt,xiq,xit,xjq,xjt)

je_sym = s.zeros(6,7)
for i, diff in enumerate([xit.b, xit.c, xit.d, xiq.a, xiq.b, xiq.c, xiq.d]):
    je_ = s.diff(e, diff)
    je_sym[ : , i] = je_

s.simplify(je_sym - je)

In [ ]:
def Je_analytic_Xj(Zq, Zt, Xiq, Xit, Xjq, Xjt):
    Jt = translation_error_derivative(Zq.conjugate()*Xiq.conjugate(), Xjt)
    JeqXq = s.Matrix([[y.b, y.c, y.d] for y in [Zq.conjugate()*Xiq.conjugate()*dpv for dpv in [s.Quaternion(x[0], x[1], x[2], x[3]) for x in np.eye(4)]]])
    print(JeqXq.shape)
    res = s.zeros(6,7)
    res[:3,:3] = Jt.T
    res[3:, 3:] = JeqXq.T

    return res

je = Je_analytic_Xj(zq,zt,xiq,xit,xjq,xjt)

je_sym = s.zeros(6,7)
for i, diff in enumerate([xjt.b, xjt.c, xjt.d, xjq.a, xjq.b, xjq.c, xjq.d]):
    je_ = s.diff(e, diff)
    je_sym[ : , i] = je_

s.simplify(je_sym - je)

$$X_i \boxplus \Delta \tilde{x} = X_{iq}*\Delta \tilde{x}_q | X_{iq}*\Delta \tilde{x}_t*X_{iq}^* + \Delta \tilde{x}_t$$

In [ ]:


def J_delta_x(Xq, Xt, dxq, dxt):
    pw = Xq.a
    pv = s.Matrix([Xq.b, Xq.c, Xq.d])
    qv = s.Matrix([dxt.b, dxt.c, dxt.d])
    
    dpvs = [
        s.Matrix([1,0,0]),
        s.Matrix([0,1,0]),
        s.Matrix([0,0,1])
    ]

    Jtt = s.Matrix([[a[0],a[1],a[2]] for a in [pv.dot(dqv)*pv + pw**2*dqv + 2*pw*pv.cross(dqv) - pv.cross(dqv).cross(pv) for dqv in dpvs]])

    dxqw = s.sqrt(1-(dxq.b**2 + dxq.c**2 + dxq.d**2))
    ddxq = [s.Quaternion(-dxq.b/dxqw,1,0,0), s.Quaternion(-dxq.c/dxqw,0,1,0), s.Quaternion(-dxq.d/dxqw,0,0,1)]

    Jq = s.Matrix([[(Xq*dq).a, (Xq*dq).b, (Xq*dq).c, (Xq*dq).d] for dq in ddxq])
    print(Jq.shape)

    res = s.zeros(7,6)
    res[:3,:3] = Jtt.T
    res[3:,3:] = Jq.T

    return res

    
        
dxq, dxt = make_symbols("\\tilde{x}_{")
dxq = s.Quaternion(s.sqrt(1-(dxq.b**2 + dxq.c**2 + dxq.d**2)), dxq.b, dxq.c, dxq.d)

Jdx = J_delta_x(xiq, xit, dxq, dxt)

et = xiq*dxt*xiq.conjugate() + xit
eq = xiq*dxq
symdx = s.zeros(7,6)
for i, diff in enumerate([dxt.b, dxt.c, dxt.d, dxq.b, dxq.c, dxq.d]):
    jet = s.diff(et, diff)
    jeq = s.diff(eq, diff)

    symdx[:3, i] = s.Matrix([jet.b, jet.c, jet.d])
    symdx[3:, i] = s.Matrix([jeq.a, jeq.b, jeq.c, jeq.d])

s.simplify(symdx - Jdx)

### Derivative of rotation wrt quaternions

In [ ]:
for symbol in [xiq.a, xiq.b, xiq.c, xiq.d]:
    print(s.simplify(s.diff(e, symbol)[5] - (zq.conjugate()*s.diff(xiq.conjugate(), symbol)*xjq).d))

for symbol in [xjq.a, xjq.b, xjq.c, xjq.d]:
    print(s.simplify(s.diff(e, symbol)[5] - (zq.conjugate()*xiq.conjugate()*s.diff(xjq, symbol)).d))

s.diff(xiq.conjugate(), xiq.b)

### Derivative of a single quaternion rotation on a pure quaternion
This is used to calculate jacobians wrt the translation component -> using the chain rule we can build up expressions for Xiq, Xit and Xjt

In [ ]:
pqp = pqp_star_pure_q(xiq, xjt)
diff = xjt.d

pw = xiq.a
pv = s.Matrix([xiq.b, xiq.c, xiq.d])
qv = s.Matrix([xjt.b, xjt.c, xjt.d])
dqv = s.diff(qv, diff)

dq = pv.dot(dqv)*pv + pw**2*dqv + 2*pw*pv.cross(dqv) - pv.cross(dqv).cross(pv)

s.simplify(s.Matrix([s.diff(pqp, diff).b, s.diff(pqp, diff).c, s.diff(pqp, diff).d]) - dq)

In [ ]:
pqp = pqp_star_pure_q(xiq, xjt)
diff = xiq.d

pw = xiq.a
pv = s.Matrix([xiq.b, xiq.c, xiq.d])
qv = s.Matrix([xjt.b, xjt.c, xjt.d])
dpv = s.diff(pv, diff)

dp = dpv.dot(qv)*pv + pv.dot(qv)*dpv + 2*pw*dpv.cross(qv) - dpv.cross(qv).cross(pv) - pv.cross(qv).cross(dpv)

s.simplify(s.Matrix([s.diff(pqp, diff).b, s.diff(pqp, diff).c, s.diff(pqp, diff).d]) - dp)

In [ ]:
pqp = pqp_star_pure_q(xiq, xjt)
diff = xiq.a

pw = xiq.a
pv = s.Matrix([xiq.b, xiq.c, xiq.d])
qv = s.Matrix([xjt.b, xjt.c, xjt.d])

dq = 2*pw*qv + 2*pv.cross(qv)

s.simplify(s.Matrix([s.diff(pqp, diff).b, s.diff(pqp, diff).c, s.diff(pqp, diff).d]) - dq)

### Composition of quaternions
This shows how to correctly combine the quaternions zq and xiq when calculating the jacobian for xjt

In [ ]:
q_comp = zq.conjugate()*xiq.conjugate()

s.simplify(q_comp*xjt*q_comp.conjugate() - zq.conjugate()*xiq.conjugate()*xjt*xiq*zq)

### Derivative of e wrt xjt & xit

In [ ]:
e = error(zq,zt,xiq,xit,xjq,xjt)
q_comp = zq.conjugate()*xiq.conjugate()


# diff = xjt.c
for diff in (xjt.b, xjt.c, xjt.d):
    pw = q_comp.a
    pv = s.Matrix([q_comp.b, q_comp.c, q_comp.d])
    qv = s.Matrix([xjt.b, xjt.c, xjt.d])
    dqv = s.diff(qv, diff)

    dq = pv.dot(dqv)*pv + pw**2*dqv + 2*pw*pv.cross(dqv) - pv.cross(dqv).cross(pv)

    de = s.diff(e, diff)
    de = s.Matrix([de[0], de[1], de[2]])

    print(s.simplify(de - dq))


In [ ]:
e = error(zq,zt,xiq,xit,xjq,xjt)
q_comp = zq.conjugate()*xiq.conjugate()

for diff in (xit.b, xit.c, xit.d):
    pw = q_comp.a
    pv = s.Matrix([q_comp.b, q_comp.c, q_comp.d])
    qv = s.Matrix([xit.b, xit.c, xit.d])
    dqv = s.diff(qv, diff)

    dq = -(pv.dot(dqv)*pv + pw**2*dqv + 2*pw*pv.cross(dqv) - pv.cross(dqv).cross(pv))

    de = s.diff(e, diff)
    de = s.Matrix([de[0], de[1], de[2]])

    print(s.simplify(de - dq))

### Derivative of et wrt xiq
This one is going to be little more complicated because its inside the composed thingy. So we're going to calculate the derivative of the interior part and then use the derivative of the interior part to calculate the derivative of the whole thing.

In [ ]:
e = error(zq,zt,xiq,xit,xjq,xjt)

for diff in (xiq.b, xiq.c, xiq.d):
    pw = xiq.a
    pv = s.Matrix([-xiq.b, -xiq.c, -xiq.d])
    qvj = s.Matrix([xjt.b, xjt.c, xjt.d])
    qvi = s.Matrix([xit.b, xit.c, xit.d])
    dpv = s.diff(pv, diff)

    dqiv = dpv.dot(qvi)*pv + pv.dot(qvi)*dpv + 2*pw*dpv.cross(qvi) - dpv.cross(qvi).cross(pv) - pv.cross(qvi).cross(dpv)
    dqjv = dpv.dot(qvj)*pv + pv.dot(qvj)*dpv + 2*pw*dpv.cross(qvj) - dpv.cross(qvj).cross(pv) - pv.cross(qvj).cross(dpv)

    qj = xiq.conjugate()*xjt*xiq
    qi = xiq.conjugate()*xit*xiq

    pw = zq.a
    pv = s.Matrix([-zq.b, -zq.c, -zq.d])
    qiv = s.Matrix([qi.b, qi.c, qi.d])
    qjv = s.Matrix([qj.b, qj.c, qj.d])

    dqi = pv.dot(dqiv)*pv + pw**2*dqiv + 2*pw*pv.cross(dqiv) - pv.cross(dqiv).cross(pv)
    dqj = pv.dot(dqjv)*pv + pw**2*dqjv + 2*pw*pv.cross(dqjv) - pv.cross(dqjv).cross(pv)

    de = s.diff(e, diff)
    de = s.Matrix([de[0], de[1], de[2]])

    print(s.simplify(de - (dqj - dqi).vec()))
    # print(s.simplify(de))

In [ ]:
pv = s.Matrix([xiq.b, xiq.c, xiq.d])
qv = s.Matrix([xjt.b, xjt.c, xjt.d])
s.diff(pv, pv[2])

## Derivative of $x_i \boxplus \Delta \tilde{x_i}$ wrt $\Delta \tilde{x_i} $

What this function does is compose a over-parameterised representation $ x_i $ with the small change in a minimally parameterised space $ \Delta \tilde{x_i} $. This should result in a $7x6$ matrix which essentially retracts the small change onto the manifold

In [ ]:
xiq, xit = make_symbols("x_{i")
dxq, dxt = make_symbols("\\tilde{x}_{")

dxq = s.Quaternion(s.sqrt(1-(dxq.b**2 + dxq.c**2 + dxq.d**2)), dxq.b, dxq.c, dxq.d)

fq = dxq*xiq
ft = dxq*xit*dxq.conjugate() - dxt

In [ ]:
s.diff(dxq, dxq.d)

In [ ]:
for diff in (dxq.b, dxq.c, dxq.d):
    pv = s.Matrix([dxq.b, dxq.c, dxq.d])
    qv = s.Matrix([xit.b, xit.c, xit.d])
    pw = s.sqrt(1-(dxq.b**2 + dxq.c**2 + dxq.d**2))
    dpv = s.diff(pv, diff)
    dpw = s.diff(pw, diff)

    dp = dpv.dot(qv)*pv + pv.dot(qv)*dpv + 2*dpw*pw*qv + 2*dpw*pv.cross(qv) + 2*pw*dpv.cross(qv)-dpv.cross(qv).cross(pv)-pv.cross(qv).cross(dpv)
    dft = s.diff(ft, diff)
    dft = s.Matrix([dft.b, dft.c, dft.d])

    display(s.simplify(dft - dp))
    # display(s.simplify(dft))
    # display(s.simplify(dp))

In [ ]:
for diff in (dxt.b, dxt.c, dxt.d):
    display(s.diff(ft, diff))
    display(s.diff(fq, diff))
    print()

In [ ]:
for diff in (dxq.b, dxq.c, dxq.d):
    dq = s.diff(dxq, diff)*xiq
    display(s.diff(fq, diff) - dq)

## Converting exponential / matrix representations to quaternion + vector


In [ ]:
exp = np.random.random(6)*2 - 1
transform = npexp(exp)

rotation = Rotation.from_matrix(transform[:3,:3]).as_quat()
translation = np.quaternion(0, *transform[:3,3])
quat = np.quaternion(rotation[-1], rotation[0], rotation[1], rotation[2])

vector = np.random.random(3)
q_v = np.quaternion(0, *vector)
h_v = np.ones(4)
h_v[:3] = vector

res_transform = transform@h_v
res_quat = quat*q_v*quat.conjugate() + translation

np.isclose(res_transform[:3], res_quat.vec)

In [ ]:
exp = np.random.random(6)*2 - 1
transform = npexp(exp)

rotation = Rotation.from_matrix(transform[:3,:3]).as_quat()
translation = np.quaternion(0, *transform[:3,3])
quat = np.quaternion(rotation[-1], rotation[0], rotation[1], rotation[2])

vector = np.random.random(3)
q_v = np.quaternion(0, *vector)
h_v = np.ones(4)
h_v[:3] = vector

res_transform = np.linalg.inv(transform)@h_v
res_quat = quat.conjugate()*q_v*quat - quat.conjugate()*translation*quat

np.isclose(res_transform[:3], res_quat.vec)

In [ ]:
Xi, Xj, Zij = np.random.random(6)*2 - 1,np.random.random(6)*2 - 1,np.random.random(6)*2 - 1
Xi, Xj, Zij = npexp(Xi), npexp(Xj), npexp(Zij)

def quat(mat):
    rotation = Rotation.from_matrix(mat[:3,:3]).as_quat()
    translation = np.quaternion(0, *mat[:3,3])
    quat = np.quaternion(rotation[-1], rotation[0], rotation[1], rotation[2])

    return quat, translation

xiq, xit = quat(Xi)
xjq, xjt = quat(Xj)
zijq, zijt = quat(Zij)

error_mat = np.linalg.inv(Zij)@np.linalg.inv(Xi)@Xj

vector = np.random.random(3)
q_v = np.quaternion(0, *vector)
h_v = np.ones(4)
h_v[:3] = vector

res_transform = error_mat

res_quat = zijq.conjugate()*xiq.conjugate()*xjt*xiq*zijq - zijq.conjugate()*xiq.conjugate()*xit*xiq*zijq - zijq.conjugate()*zijt*zijq

np.isclose(res_transform[:3, 3], res_quat.vec)

res_transform_quat = np.roll(Rotation.from_matrix(res_transform[:3,:3]).as_quat(), 1)

if not np.all(np.isclose(res_transform_quat, np.array((zijq.conjugate()*xiq.conjugate()*xjq).components))) and not np.all(np.isclose(res_transform_quat*-1, np.array((zijq.conjugate()*xiq.conjugate()*xjq).components))):
    print(res_transform_quat, np.array((zijq.conjugate()*xiq.conjugate()*xjq).components))
# np.isclose(res_transform_quat, np.array((zijq.conjugate()*xiq.conjugate()*xjq).components))

In [ ]:
exp = np.random.random(6)*2 - 1
transform = npexp(exp)

exp2 = np.random.random(6)*2 - 1
transform2 = npexp(exp)

rotation = Rotation.from_matrix(transform[:3,:3]).as_quat()
translation = np.quaternion(0, *transform[:3,3])
quat = np.quaternion(rotation[-1], rotation[0], rotation[1], rotation[2])

rotation2 = Rotation.from_matrix(transform2[:3,:3]).as_quat()
translation2 = np.quaternion(0, *transform2[:3,3])
quat2 = np.quaternion(rotation2[-1], rotation2[0], rotation2[1], rotation2[2])



res_transform = transform@transform2
res_quat = quat*quat2
res_quat_translation = quat*translation2*quat.conjugate() + translation

res_quat_total = np.eye(4)
res_quat_total[:3,:3] = Rotation.from_quat([res_quat.x, res_quat.y, res_quat.z, res_quat.w]).as_matrix()
res_quat_total[:3, 3] = res_quat_translation.vec

np.isclose(res_transform, res_quat_total)

## Convert Exponential to Quaternion
Should be able to just pull the translation from the matrix representation of the 

In [ ]:
exp = np.random.random(6)*2 - 1
transform = npexp(exp)

angle = np.linalg.norm(exp[3:])
axis = exp[3:]/angle

quat = np.quaternion(np.cos(angle/2), *(np.sin(angle/2)*axis))
np.isclose(np.array(quat.components), np.roll(Rotation.from_matrix(transform[:3,:3]).as_quat(), 1))